In [13]:
import pandas as pd
sub_mission=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\카글\house-prices-advanced-regression-techniques\sample_submission.csv")
test=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\카글\house-prices-advanced-regression-techniques\test.csv")
train=pd.read_csv(r"C:\Users\SAMSUNG\Desktop\공부파일들\카글\house-prices-advanced-regression-techniques\train.csv")

In [10]:
import sklearn.preprocessing as skpre
import sklearn.model_selection as skmod
import sklearn.pipeline as skpip
import sklearn.compose as skcom
import sklearn.impute as skimp
import numpy as np

In [14]:
train.drop('Id',inplace=True,axis=1)
num_col=train.select_dtypes(np.number).columns.difference(['SalePrice'])
cate_col=train.select_dtypes("object").columns

In [15]:
# 파이프라인
num_pip=skpip.make_pipeline(skimp.SimpleImputer(strategy="median"),
                            skpre.StandardScaler())
cate_pip=skpip.make_pipeline(skimp.SimpleImputer(strategy="most_frequent"),
                             skpre.OneHotEncoder(handle_unknown="ignore"))

# 합치기
col_pip=skcom.make_column_transformer((num_pip,num_col),
                                      (cate_pip,cate_col))
# 결과
pip_result=col_pip.fit_transform(train)
pre_train=pd.DataFrame(pip_result.toarray(),columns=col_pip.get_feature_names_out())

### 그리드 서치의 사용법

In [33]:
# 랜덤시드
np.random.seed(10)
# 다른 모델을 하나 만들겠음
import sklearn.neighbors as sknei
knn_regression=sknei.KNeighborsRegressor(n_jobs=-1)
total_pip=skpip.make_pipeline(col_pip,knn_regression)

In [34]:
# 변환기 이름 확인
total_pip.named_steps.keys()

dict_keys(['columntransformer', 'kneighborsregressor'])

In [32]:
knn_regression.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [73]:
# 그리드서치 확인
params=[{"kneighborsregressor__n_neighbors":[3,5,7,9],"kneighborsregressor__p":[2]}]
       # {"kneighborregressor__"}] 이처럼 리스트에 다른 조합의 grid를 넣으면 추가로 계산함
grid_search=skmod.GridSearchCV(estimator=total_pip,cv=10,n_jobs=-1,
                                param_grid=params,scoring="neg_root_mean_squared_error",return_train_score=True)

In [74]:
grid_search.fit(X=train,y=train["SalePrice"])

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('pipeline-1',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'Enclosed...
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                                       ('kneighborsregressor',
                                        KNeighborsRegressor(n_jobs=-1))]),
             n_jobs=-1,
             param_grid=[{'kneighborsregressor__n_neighbors': [3, 5, 7, 9],
                          'kneighborsregressor__p': [2]}],
             return_train_score=True, scoring='neg_root_mean_squared_error')

### 주의
위와 같이 `변환기이름__파라미터명` 규칙을 따라야 해당변환기의 파라미터를 찾는다 

In [79]:
# 결과보기
result=pd.DataFrame(grid_search.cv_results_).dropna(axis=0)
cv_result=result[['param_kneighborsregressor__n_neighbors', 'param_kneighborsregressor__p'
        ,'rank_test_score','mean_train_score']].sort_values(by="mean_train_score",ascending=False)
cv_result["mean_rmse"]=-cv_result["mean_train_score"]
cv_result

,param_kneighborsregressor__n_neighbors,param_kneighborsregressor__p,rank_test_score,mean_train_score,mean_rmse
0,3,2,4,-26553.036705,26553.036705
1,5,2,1,-31370.987220,31370.987220
2,7,2,3,-33342.689112,33342.689112
3,9,2,2,-34121.372611,34121.372611


비교해보자

In [80]:
total_pip.fit(X=train,y=train["SalePrice"])

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('kneighborsregressor', KNeighborsRegressor(n_jobs=-1))])

In [81]:
import sklearn.metrics as skmet

In [85]:
np.sqrt(skmet.mean_squared_error(y_true=train["SalePrice"],y_pred=total_pip.predict(train)))

31243.05168550303

왜 `best params`가 `5,2`인지 이상하다.
실제로 위의 표를 보았을땐 `3,2`가 `rmse`값이 가장 낮게 나온다

랜덤서치는 위와 동일하므로 스킵